In [ ]:
!pip install 

In [4]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from dotenv import load_dotenv

import os, time, uuid
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

load_dotenv()

True

In [5]:
CONFIG = {
    'TRAINING_ENDPOINT' : os.getenv('OBJECT_DETECTION_TRAINING_ENDPOINT'), 
    'TRAINING_KEY' : os.getenv('OBJECT_DETECTION_TRAINING_KEY'),
    'TRAINING_RESOURCE_ID' : os.getenv('OBJECT_DETECTION_TRAINING_RESOURCE_ID'),

    'PREDICTION_ENDPOINT' : os.getenv('OBJECT_DETECTION_PREDICTION_ENDPOINT'),
    'PREDICTION_KEY' : os.getenv('OBJECT_DETECTION_PREDICTION_KEY'),
    'PREDICTION_RESOURCE_ID' : os.getenv('OBJECT_DETECTION_PREDICTION_RESOURCE_ID')
}

In [ ]:
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [ ]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": CONFIG['TRAINING_KEY']})
trainer = CustomVisionTrainingClient(CONFIG['TRAINING_ENDPOINT'], training_credentials)

### Model Prediction

In [ ]:
publish_iteration_name = "udacity-2-classes-object-detection-custom"
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

In [ ]:
local_image_path = '/home/workspace'
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project.id, publish_iteration_name, image_contents.read())
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [ ]:
file_name = "flower.jpg"
perform_prediction(file_name)
## Checking the Image
with open(os.path.join (local_image_path, file_name), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

### Exporting Model

In [ ]:
platform = "TensorFlow"
flavor = "TensorFlowLite"
export_process = trainer.export_iteration(project.id, iteration.id, platform, flavor, raw=True)

In [ ]:
print(export_process.output)

In [ ]:
print(export_process.output.status)

In [ ]:
### Code snippet is from Azure SDK and Documentation
### https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
### This step may take long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project_id, selected_iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

In [ ]:
print(export_process.output.status)
print(export_process.output.download_uri)

In [ ]:
## Downloading the model from url
if export_process.output.status == "Done":
    # Ready to Download
    model_export_file = requests.get(export_process.output.download_uri)
    with open("od_model_tensorflow.zip", "wb") as file:
        file.write(model_export_file.content)